## Question:
What makes a quote go viral?

## Terminology:
**VIRAL:** more than 100 occurrences on different sites.

## Application:
Providing insight on how politicians, influencers, etc. may obtain lots of visibility on a single quote.

## Outcome Variables:
- Viral: yes / no
- How fast viral viral: in how long viral quotes reached 2/3 of occurrences.

## Features:
- Indicator variables for 3 most common jobs
- Indicator variables for 3 most common genders
- Indicator variables for 3 most common ethnicities
- Indicator variables for 3 most common age of speaker
- Hour of quote?
- Topic: detect most common topics and indicator vars of them

## Technique:
Linear regression / Logistic regression / SVM